# Testing the Model

**Download the normal model here: https://drive.google.com/open?id=1-5zaLocjGkIn7ZwsieCcnmdJw04JsNnp**

**Download the EC model here: https://drive.google.com/file/d/1cqDoyR7i74Qc2leFKjcutmxWP38dehER**

### Enter the filepaths for the data, labels, and model

In [1]:
dataDir = './test_data.npy'

In [2]:
labelsDir = './test_labels.npy'

In [3]:
load_model_filepath = './best_model.h5'

In [4]:
load_model_filepath_ec = './model_ec.h5'

### Once you've entered the file paths, run all and go to bottom of file to see results

In [5]:
from matplotlib import pyplot as plt
import numpy as np
import collections

from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model

from sklearn.metrics import accuracy_score

import seaborn as sns

In [6]:
class Classifier:
    def __init__(self,numClasses, input_shape, batch_size, epochs):
        # with strategy.scope():
            self.numClasses = numClasses
            self.input_shape = input_shape
            self.batch_size = batch_size
            self.epochs = epochs
            

            self.model = Sequential()

            self.model.add(Conv2D(64,kernel_size=2,
                            activation='relu',
                            padding= 'same',
                            input_shape=input_shape, data_format='channels_last'))

            self.model.add(Conv2D(128, (3, 3), activation='relu'))
            self.model.add(MaxPooling2D(pool_size=(2, 2)))
            
            self.model.add(Conv2D(128, (3, 3), activation='relu'))
            self.model.add(MaxPooling2D(pool_size=(2, 2)))

            self.model.add(Dropout(.5))
            
            self.model.add(Flatten())

            self.model.add(Dense(128, activation='relu'))
            self.model.add(Dropout(.5))
            
            self.model.add(Dense(self.numClasses, activation='softmax'))



            sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
            adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

            
            self.model.compile(loss='categorical_crossentropy',
                          optimizer= adam,
                          metrics=['accuracy'])
            
            self.model.summary()


    def Train(self, Xtrain, ytrain, Xtest, ytest, model_filepath):
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
        mc = ModelCheckpoint(model_filepath, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
        self.model.fit(Xtrain, ytrain,
                    steps_per_epoch=len(Xtrain)//self.batch_size, 
                    validation_data= (Xtest, ytest), 
                    validation_steps=len(Xtest)//self.batch_size,
                    epochs=self.epochs,
                    shuffle=True,
                    verbose = 1, 
                    callbacks= [es, mc])
          
    def Test(self, Xtest, ytest): 
        if('A' in ytest):
           ytest = keras.utils.to_categorical(charToInt(ytest));
       
        predictions = self.model.predict(Xtest)
        y_pred = np.argmax(predictions, axis=1)
        ytest = ytest.argmax(axis=1)

        ACC = accuracy_score(ytest, y_pred)
        print('\nClassification Accuracy (ACC): {0:0.1f}'.format(ACC*100) + '%')

        print("\nConfusion Matrix")                                          
        cm = confusion_matrix(ytest, y_pred)

        sns.heatmap(cm, cbar=False, annot=True,fmt="d", linewidths=.5,
              yticklabels=['A','B','C','D','E','F','G','H','I'],
              xticklabels=['A','B','C','D','E','F','G','H','I'])
        plt.xlabel('True label')
        plt.ylabel('Predicted label')
        plt.show()

        print(classification_report(y_pred, ytest))

    def TestEC(self, Xtest, ytest):        
        if('A' in ytest):
            ytest = keras.utils.to_categorical(charToInt(ytest));
       
        predictions = self.model.predict(Xtest)
        y_pred = np.argmax(predictions, axis=1)
        ytest = ytest.argmax(axis=1)

        ACC = accuracy_score(ytest, y_pred)
        print('\nClassification Accuracy (ACC): {0:0.1f}'.format(ACC*100) + '%')

        print("\nConfusion Matrix")                                          
        cm = confusion_matrix(ytest, y_pred)

        sns.heatmap(cm, cbar=False, annot=True,fmt="d", linewidths=.5,
              yticklabels=['unknown','A','B','C','D','E','F','G','H','I'],
              xticklabels=['unknown','A','B','C','D','E','F','G','H','I'])
        plt.xlabel('True label')
        plt.ylabel('Predicted label')
        plt.show()

        print(classification_report(y_pred, ytest))


    

    def SaveModel(self, filename):
        self.model.save(filename)
    
    def LoadModel(self, filename):
        self.model = load_model(filename)

    def Plot(self):
        plot_model(self.model)


In [7]:
def charToInt(data):
    charToIntdict = {'A': 1, 'B': 2,'C': 3, 'D': 4,'E': 5,'F': 6,'G' : 7, 'H': 8, 'I': 9, 'unknown': -1}
    temp = np.zeros(len(data))
    for index, element in enumerate(data):
        temp[index] = charToIntdict[element[0]]
    return temp

In [9]:
from sklearn.model_selection import train_test_split

data = np.load(dataDir)
labels = np.load(labelsDir)

Xtest = data.astype(np.float32) / 255.0
ytest = keras.utils.to_categorical(charToInt(labels))

print(Xtest[0].dtype)
Xtest.shape, ytest.shape

float32


((1844, 100, 100, 3), (1844, 10))

In [10]:
numClasses = 10
input_shape =(100,100,3)
batch_size = 64
epochs=130

best_model = Classifier(numClasses=numClasses, input_shape=input_shape, batch_size=batch_size, epochs=epochs)
best_model.LoadModel(load_model_filepath)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 64)      832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 98, 98, 128)       73856     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 67712)             0

In [ ]:
best_model.Test(Xtest, ytest)

C:\Users\frank\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
